<a href="https://colab.research.google.com/github/cdancette/deep-learning-polytech-tp6-7/blob/master/TP_10_GAN_Deep_Learning_Polytech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Avant de commencer : 
Fichier -> Enregistrer une copie dans Drive

In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

In [0]:
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

## Hyperpameters

Define the hyperparameters. You can play with those later.

In [0]:
## Data loading

workers = 0 # Number of workers for dataloader (/!\ set to 4 when you're done debugging)

## Architecture

nz = 100 # Size of z latent vector (i.e. size of generator input)
ndf = 32 # Base size of feature maps in discriminator
ngf = 32 # Base size of feature maps in generator

## Optimization

lrD = 0.0002 # Learning rate for the discriminator
lrG = 0.0002 # Learning rate for the generator
beta1G = 0.5 # Momentum beta1 for the discriminator
beta1D = 0.5 # Momentum beta1 for the generator

## Training

batch_size = 256 # Images per batch
nb_update_D = 1 # Number of sub-steps of discriminator optim. at each step
nb_update_G = 1 # Number of sub-steps of generator optim. at each step
steps = 8000 # Number of global steps in the training loop
nb_epochs = None # Number of epochs, leave "None" if you want to set the number of "steps" (i.e. batches)


if nb_epochs is None:
    nb_epochs = (steps * batch_size) / (nb_update_D * 202000)
else:
    steps = int(nb_epochs * nb_update_D * 202000 / batch_size)
print("Doing %.1f epochs in %d steps" % (nb_epochs, steps))

Doing 10.1 epochs in 8000 steps


# Dataset

Download and load the dataset. Nothing to do here.

In [0]:
%%sh
# Dataset loading

if ! [ -d "/tmp/celeba/img_align_celeba" ] ; then
    mkdir /tmp/celeba
    cd /tmp/celeba
    wget http://webia.lip6.fr/~robert/cours/rdfia/celeba.zip
    unzip celeba.zip
fi

# For 64x64 images, replace celeba with celeba64 everywhere

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

--2019-11-28 11:44:03--  http://webia.lip6.fr/~robert/cours/rdfia/celeba.zip
Resolving webia.lip6.fr (webia.lip6.fr)... 132.227.201.33
Connecting to webia.lip6.fr (webia.lip6.fr)|132.227.201.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153357646 (146M) [application/zip]
Saving to: ‘celeba.zip’

     0K .......... .......... .......... .......... ..........  0%  247K 10m7s
    50K .......... .......... .......... .......... ..........  0%  492K 7m35s
   100K .......... .......... .......... .......... ..........  0% 44.1M 5m5s
   150K .......... .......... .......... .......... ..........  0% 73.9M 3m49s
   200K .......... .......... 

In [0]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root="/tmp/celeba",
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
plt.show()

# Model architectures

## Discriminator

Input: Image $x \in \mathbb{R}^{32\times 32\times 3}$  
Output: "Real" image probability $\in [0,1]$

* Convolution (`1*ndf` filters, kernel 4, stride 2, padding 1, no bias)
* Batch Norm 2D
* LeakyReLU ($\alpha = 0.2$)
* Convolution (`2*ndf` filters, kernel 4, stride 2, padding 1, no bias)
* Batch Norm 2D
* LeakyReLU ($\alpha = 0.2$)
* Convolution (`4*ndf` filters, kernel 4, stride 2, padding 1, no bias)
* Batch Norm 2D
* LeakyReLU ($\alpha = 0.2$)
* Convolution (1 filter, kernel 4, stride 1, padding 0, no bias)
* Sigmoid activation

## Generator

Input: Random "noise" $z \in \mathbb{R}^{\text{nz}}$  
Output: Generated image $\tilde x \in \mathbb{R}^{32\times 32\times 3}$

* Convolution Transpose (`4*ngf` filters, kernel 4, stride 1, padding 0, no bias)
* Batch Norm 2D
* ReLU
* Convolution Transpose (`2*ngf` filters, kernel 4, stride 2, padding 1, no bias)
* Batch Norm 2D
* ReLU
* Convolution Transpose (`1*ngf` filters, kernel 4, stride 2, padding 1, no bias)
* Batch Norm 2D
* ReLU
* Convolution Transpose (3 filters, kernel 4, stride 2, padding 1, no bias)
* Tanh activation

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # TODO
        )

    def forward(self, input):
        return self.model(input)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # TODO
        )

    def forward(self, input):
        return self.model(input)

In [0]:
# Create the models
netG = Generator().to(device)
netD = Discriminator().to(device)

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
netG.apply(weights_init)
netD.apply(weights_init)

# Print the models
print(netG)
print(netD)

In [0]:
# TODO test you models to check if they 
z = torch.zeros(10, nz, 1, 1).to(device)
x = torch.zeros(10, 3, 32, 32).to(device)
print(netG(z).shape) # expected: [10, 3, 32, 32]
print(netD(x).shape) # expected: [10, 1, 1, 1]

# Optimization and training

Here we will define:
* Our prior $P(z)$ that we use to sample random "noise". We will use a Gaussian distribution.
* The criterion that will be used to train the discriminator, and indirectly the generator. We will use the binary cross-entropy.
* The optimizers of both models. We will use the Adam optimizer.

In [0]:
# Prior P(z). Returns a Gaussian random tensor of shape (batch_size, nz, 1, 1)
def get_noise(batch_size):
    noise = None # TODO
    return noise.to(device)

# Create the criterion function that will take (y_hat, y) as input
criterion = None # TODO

# Setup Adam optimizers for D and G
optimizerD = None # TODO, take netD.parameters(), use the right lr and beta1
optimizerG = None # TODO, same for G

### Data format / batch creation functions

In [0]:
# Data format / batch creation functions

fixed_noise = get_noise(196) # Create a fixed random vector sampled from a Gaussian, will be used during train for viz
real_label = 1
fake_label = 0
iterator = iter(dataloader)

# returns a batch of real images from the dataset (iterates infinitely on the dataset)
def get_batch_real():
    global iterator
    try:
        x_real = next(iterator)[0].to(device)
    except:
        iterator = iter(dataloader)
        x_real = next(iterator)[0].to(device)
    y_real = torch.full((x_real.size(0),), real_label, device=device)
    return x_real, y_real

# TODO
# returns a batch of generated images and training targets y_fake
# Note that the targets y_fake will be different is train_G is True or False
def get_batch_fake(train_G=False):
    z = get_noise(batch_size)
    x_fake = None # TODO generate images from z
    y_fake = None # TODO create targets, depends on train_G
    return x_fake, y_fake

### Training Loop

**Reminder:** when your training loop starts to work, change the `workers` variable to 4 and rerun your notebook

In [0]:
img_list = []
G_losses = []
D_losses = []

In [0]:
for i in range(steps):
    
    ########
    # Update D network: maximize log(D(x)) + log(1 - D(G(z)))
    for _ in range(nb_update_D):
        netD.zero_grad()
        
        # Create batches
        x_real, y_real = get_batch_real()
        x_fake, y_fake = get_batch_fake()
        
        # Forward 
        y_hat_real = None # TODO
        y_hat_fake = None # TODO
        errD = None       # TODO sum of criterion of real and fake samples
        
        # Backward
        # TODO backward & optimization step on D
        
        # Compute / save metrics
        avg_output_for_real = y_hat_real.mean().item()
        avg_output_for_fake = y_hat_fake.mean().item()    
        D_losses.append(errD.item())



    ########
    # Update G network: maximize log(D(G(z)))
    for _ in range(nb_update_G):
        netG.zero_grad()
        
        # TODO: forward + backward
        # NOTE: use errG as name for your loss variable, like errD above
        
        # Compute / save metrics
        G_losses.append(errG.item())
        
    
    ########
    # Logs
    if i % 25 == 0:
        print('[%5d/%5d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f'
              % (i, steps, errD.item(), errG.item(), avg_output_for_real, avg_output_for_fake))

    if i % 100 == 0:
        with torch.no_grad():
            x_fake = netG(fixed_noise).detach().cpu()
        img_list.append(vutils.make_grid(x_fake, padding=2, normalize=True, nrow=14))
        plt.figure(figsize=(8,8))
        plt.imshow(np.transpose(img_list[-1],(1,2,0)))
        plt.axis("off")
        plt.show()

# Display training evolution

In [0]:
# Show generations
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [0]:
# Loss evolution
plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("Generator Trainig Loss")
plt.plot(G_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")

plt.subplot(2,1,2)
plt.title("Generator Trainig Loss")
plt.plot(D_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.show()